# Preprocessin for Sentiment Analysis
> * [공공데이터포털](https://www.data.go.kr/) : 뉴스빅데이터 분석 정보
> * 수정일 2017.12.26 

In [1]:
# -*- coding: utf-8 -*-

In [2]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import re

In [3]:
path1 = './data/news/have_negative_positive/constitution/'
path2 = './data/news/have_negative_positive/household_debt/'
path3 = './data/news/have_negative_positive/olymphic/'
path4 = './data/news/have_negative_positive/'
pathList = [path1, path2, path3, path4]

In [4]:
def Get_CSVfileList(path):
    filelist = glob(path+'*.csv')
    filelist.extend(glob(path+'*.CSV'))
    return filelist

In [5]:
df_data=pd.DataFrame()
x = 0 
for path in pathList:
    fileList = Get_CSVfileList(path)
    for file in fileList:
        df = pd.read_csv(file,encoding='utf-8', engine='python')
        x += df.shape[0]
        if len(df_data) == 0:
            df_data = df[['감성분석', '긍정 문장', '부정 문장']]
        else:
            df_data = pd.concat([df_data, df[['감성분석', '긍정 문장', '부정 문장']]])
df_data.reset_index(drop=True, inplace=True)        
print (x)
print (df_data.shape)
df_data.head()

176643
(176643, 3)


,감성분석,긍정 문장,부정 문장
0,NEGATIVE 폭압 NEGATIVE 사이비 NEGATIVE 족쇄 NEGATIVE ...,주택조합 승인 자체가 몹시 어려웠던 시절 한국기자협회 주택조합 형식으로 추진된 이 ...,[도청도설] 기자 아파트 단독주택 420여 가구를 짓는 이 사업은 꼭 5년이 걸려 ...
1,NEGATIVE 문제,"서 의원은 여의도 당사에서 기자회견을 열어 “압도적으로 승리할 것이고 국민, 당원이...",김 의원은 여의도 당사 기자회견에서 “대통령이 와서 지난 대선 때 도와준 핵심 간부...
2,NEGATIVE 제외 NEGATIVE 의혹 NEGATIVE 비리 NEGATIVE 포...,박근혜 대통령이 14일 열린 새누리당 전당대회에 참석한 것은 그만큼 아직 여권에서 ...,실제로 1987년 이후 역대 대통령들은 집권 초에는 여당에 막강한 영향력을 행사했지...
3,POSITIVE 최선 POSITIVE 열심히 NEGATIVE 비판 NEGATIVE ...,"“살기 좋은 전북 만들기, 중앙정부와 연결고리 최선” “월드컵에 출전한 우리 국가대...","‘장기 비전이 없다면, 살아남을 수 없다’, ‘플랜B(위기 대응책)없이는 살아남을 ..."
4,POSITIVE 고맙 NEGATIVE 탈락 NEGATIVE 어렵 POSITIVE 구...,"""여러분 너무나 고맙고 감사합니다. 도와주신 여러분께 저의 마음속의 큰 절을 받아주...",공천 탈락 직후 백의종군한 김무성 당시 선대본부장이 당을 떠나며 말없이 남긴 석 줄...


In [6]:
df_words = df_data['감성분석'].copy()
df_positive = df_data['긍정 문장'].copy()
df_negative = df_data['부정 문장'].copy()

* drop list
> 기자 & 이메일 주소만 있는 문장  
> Neutral 제거

## 긍정 문장 추출

### NaN & ' '제거

In [7]:
df_positive.dropna(inplace = True)
df_positive = df_positive.apply(html.unescape)
df_positive.reset_index(drop=True, inplace = True)
df_positive.drop(df_positive[df_positive.str.match('^ $')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156189,)


### #*****? 인 값 제거

In [8]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156116,)


### #*****! 인 값 제거

In [9]:
df_positive.drop(df_positive[df_positive.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(156089,)


### 이메일만 있는 값 제거

In [10]:
df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(155267,)


### 이메일과 기자명만 있는 값 제거

In [11]:
df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[df_positive.str.len() < 11].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

df_positive.drop(df_positive[(df_positive.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_positive.str.len() < 100)].index, inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

print (df_positive[df_positive==''])

df_positive = df_positive.apply(html.unescape)

df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)
df_positive = df_positive.apply(html.unescape)
df_positive.drop_duplicates(inplace = True)
df_positive.reset_index(drop=True, inplace = True)
print (df_positive.shape)

(154553,)
(154129,)
(154102,)
(154082,)
(154052,)
(154035,)
(153769,)
(153762,)
(153655,)
(153500,)
(153100,)
(152853,)
(151401,)
(151272,)
Series([], Name: 긍정 문장, dtype: object)
(144397,)
(144397,)


In [12]:
df_positive.to_csv('./data/sentiment_data/positive_sentiment_sentence_from_news.csv', index=False, header= False, encoding='utf-8')

## 부정 문장 추출

### NaN & ' '제거

In [13]:
df_negative.dropna(inplace = True)
df_negative = df_negative.apply(html.unescape)
df_negative.reset_index(drop=True, inplace = True)
df_negative.drop(df_negative[df_negative.str.match('^ $')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160872,)


### #*****? 인 값 제거

In [14]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### #*****! 인 값 제거

In [15]:
df_negative.drop(df_negative[df_negative.str.match('^#[a-zA-Z0-9]+\!$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### 이메일만 있는 값 제거

In [16]:
df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')].index,inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160832,)


### 이메일과 기자명만 있는 값 제거

In [17]:
df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*$')].index , inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[가-힣]{3,3}[. ]기자[. ][a-zA-Z0-9+-_.]+@[a-zA-Z0-9-. ]*$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[. ][가-힣]{3,3}기자')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_. ]+[가-힣 ]{3,4}기자$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[a-zA-Z0-9+-_.]+@([a-zA-Z0-9-]+\.[a-zA-Z0-9-.])*')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('.[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.match('[가-힣+_.=]+?[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\|]*?$')].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[df_negative.str.len() < 11].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

df_negative.drop(df_negative[(df_negative.str.match('^[가-힣]{3,5}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-.\| @]*?')) & (df_negative.str.len() < 100)].index, inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

print (df_negative[df_negative==''])

df_negative = df_negative.apply(html.unescape)

df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)
df_negative = df_negative.apply(html.unescape)
df_negative.drop_duplicates(inplace = True)
df_negative.reset_index(drop=True, inplace = True)
print (df_negative.shape)

(160612,)
(160558,)
(160543,)
(160536,)
(160510,)
(160227,)
(160140,)
(160140,)
(159943,)
(159863,)
(159699,)
(159500,)
(158488,)
(158326,)
Series([], Name: 부정 문장, dtype: object)
(151957,)
(151957,)


In [18]:
df_negative.to_csv('./data/sentiment_data/negative_sentiment_sentence_from_news.csv', index=False, header=False, encoding='utf-8')

## 긍정 / 부정 단어 추출

In [19]:
df_words.dropna(inplace = True)
df_words = df_words.apply(html.unescape)
df_words.reset_index(drop = True, inplace = True)
df_words.drop(df_words[df_words.str.match('^ $')].index, inplace = True)
df_words.reset_index(drop = True, inplace = True)
df_words = df_words.apply(html.unescape)
print(df_words.shape)

(167087,)


In [20]:
df_words = df_words[df_words.str.match('NEGATIVE|POSITIVE')]
df_words.reset_index(drop = True, inplace = True)
print (df_words.shape)

(164046,)


In [21]:
negativeWords = df_words.str.findall('(NEGATIVE [가-힣]+)').str.join(' ')
negativeWords.dropna(inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
positiveWords = df_words.str.findall('(POSITIVE [가-힣]+)').str.join(' ')
positiveWords.dropna(inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)

(164046,) (164046,)


In [22]:
positiveWords.drop(positiveWords[positiveWords==''].index, inplace = True)
positiveWords.reset_index(drop=True, inplace = True)
negativeWords.drop(negativeWords[negativeWords==''].index, inplace = True)
negativeWords.reset_index(drop=True, inplace = True)
print (negativeWords.shape, positiveWords.shape)
positiveWords = positiveWords.str.findall('([가-힣]+)')
negativeWords = negativeWords.str.findall('([가-힣]+)')

(144425,) (123623,)


In [23]:
positiveWords = list(set(itertools.chain.from_iterable(positiveWords)))
negativeWords = list(set(itertools.chain.from_iterable(negativeWords)))
print (len(positiveWords), len(negativeWords))

5010 7608


In [25]:
positiveWords = pd.Series(positiveWords)
negativeWords = pd.Series(negativeWords)

In [29]:
positiveWords.to_csv('./data/sentiment_data/positive_sentiment_word_from_news.csv', index=False, header=False)
negativeWords.to_csv('./data/sentiment_data/negative_sentiment_word_from_news.csv', index=False, header=False)

In [30]:
df_positive = pd.concat([df_positive, pd.DataFrame(len(df_positive)*[1], columns=[1])],axis = 1)
df_positive.rename(columns={'긍정 문장':0},inplace = True)
df_negative = pd.concat([df_negative, pd.DataFrame(len(df_negative)*[0], columns=[1])],axis = 1)
df_negative.rename(columns={'부정 문장':0},inplace = True)


In [31]:
newsSentence = pd.concat([df_positive, df_negative], axis = 0)
newsSentence.reset_index(drop=True, inplace = True)
print (newsSentence.shape)
newsSentence.head()

(296354, 2)


,0,1
0,주택조합 승인 자체가 몹시 어려웠던 시절 한국기자협회 주택조합 형식으로 추진된 이 ...,1
1,"서 의원은 여의도 당사에서 기자회견을 열어 “압도적으로 승리할 것이고 국민, 당원이...",1
2,박근혜 대통령이 14일 열린 새누리당 전당대회에 참석한 것은 그만큼 아직 여권에서 ...,1
3,"“살기 좋은 전북 만들기, 중앙정부와 연결고리 최선” “월드컵에 출전한 우리 국가대...",1
4,"""여러분 너무나 고맙고 감사합니다. 도와주신 여러분께 저의 마음속의 큰 절을 받아주...",1


## 특수문자 & 기자명 & 이메일 제거 및 50자 이내 제거

In [32]:
newsSentence[0] = newsSentence[0].str.replace('^[가-힣]{3,4}[//. ]?기자[//. ]',' ')

newsSentence[0] = newsSentence[0].str.replace('^[ ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]*',' ')

newsSentence[0] = newsSentence[0].str.replace('[가-힣]{3,4}[//. ]?기자[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',' ')
print (newsSentence.shape)

newsSentence[0] = newsSentence[0].str.replace('[가-힣]{3,4}[//. ]?기자',' ')

newsPress = pd.read_csv('./data/newspress.txt',encoding='utf-8',header=None)
newsPress.head()

for idx in newsPress.index:
    newsSentence[0] = newsSentence[0].str.replace(newsPress.loc[idx][0],' ')

newsSentence[0] = newsSentence[0].str.replace('[☜\[\]☞※©◀◆◁◀◈○◌◎●◑◐☆★\|♠♡〈〉『」「』▲=⑨①②ⓒ③「」◎○~!@#$%^&*()_+|<>?:{}]', ' ')

newsSentence[0] = newsSentence[0].str.replace('[//. ]?[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]*','')

newsSentence[0] = newsSentence[0].str.replace('[a-zA-Z0-9+-.]+@','')

newsSentence[0] = newsSentence[0].str.strip()

newsSentence.drop(newsSentence[newsSentence[0].str.len()<10].index, inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.dropna(inplace=True)
print (newsSentence.shape)

newsSentence.drop(newsSentence[newsSentence[0].str.len()<50].index, inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.dropna(inplace=True)
newsSentence.reset_index(drop=True, inplace=True)
newsSentence.drop_duplicates(inplace = True)
newsSentence.reset_index(drop=True, inplace=True)
print (newsSentence.shape)
newsSentence.to_csv('./data/sentiment_data/merged_sentiment_data.txt',index=False, header=False)
newsSentence.head()

(296354, 2)
(295847, 2)
(278900, 2)


,0,1
0,주택조합 승인 자체가 몹시 어려웠던 시절 한국기자협회 주택조합 형식으로 추진된 이 ...,1
1,"서 의원은 여의도 회견을 열어 “압도적으로 승리할 것이고 국민, 당원이 주문하신 ...",1
2,박근혜 대통령이 14일 열린 새누리당 전당대회에 참석한 것은 그만큼 아직 여권에서 ...,1
3,"“살기 좋은 전북 만들기, 중앙정부와 연결고리 최선” “월드컵에 출전한 우리 국가대...",1
4,"""여러분 너무나 고맙고 감사합니다. 도와주신 여러분께 저의 마음속의 큰 절을 받아주...",1


In [33]:
newsSentence = pd.read_csv('./data/sentiment_data/raw_data_sentiment.txt',header=None,encoding='utf-8')
print (newsSentence.shape)
newsSentence.head()

(291577, 2)


,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


In [34]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header
    return data
rating = pd.DataFrame(read_data('./data/sentiment_data/ratings.txt'))
rating.drop(0, axis = 1, inplace = True)
rating.rename(columns = {1:0, 2:1}, inplace = True)
print (rating.shape)
rating.head()

(200000, 2)


,0,1
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [35]:
totalData = pd.concat([newsSentence, rating])
totalData.reset_index(drop=True, inplace = True)
totalData.dropna(inplace=True)
print (totalData.shape)
totalData.reset_index(drop=True, inplace = True)
totalData.drop(totalData[totalData[0] ==''].index, inplace = True)
print (totalData.shape)
totalData.reset_index(drop=True, inplace = True)
totalData.to_csv('./data/sentiment_data/merged_rawdata_rating.txt',index=False, header=False, encoding='utf-8')
print (totalData.shape)
totalData.head()

(491577, 2)
(491569, 2)
(491569, 2)


,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1
